# hashlib摘要 和 加密与解密

## hash摘要算法

hashlib是个专门提供hash算法的库，现在包括md5, sha1, sha224, sha256, sha384, sha512，使用非常简单。

* md5经常用来做用户密码的存储。
* sha1则经常用作数字签名。

md5生成一个128bit的结果，通常用32位的16进制字符串表示 

sha1生成一个160bit的结果，通常用40位的16进制字符串表示

SHA256和SHA512，不过越安全的算法越慢，而且摘要长度更长

注意：
* 哈希算法又称为摘要算法，它生成的是固定长度的不可逆的混杂字符串。
* 不要与加密算法混为一谈，因为加密对应着解密。而哈希算法很难很难解密。

### MD5摘要

In [4]:
import hashlib

def md5_crypto(pwd):
    md_5 = hashlib.md5()
    md_5.update(pwd.encode('utf8'))
    md5_pwd = md_5.hexdigest()
    return md5_pwd

pwd = md5_crypto('123456')
print(len(pwd))
print(pwd)

32
e10adc3949ba59abbe56e057f20f883e


### sha1摘要

In [3]:
import hashlib

def sha1_crypto(pwd):
    sha1 = hashlib.sha1()
    sha1.update(pwd.encode('utf8'))
    sha1_pwd = sha1.hexdigest()
    return sha1_pwd

pwd = sha1_crypto('123456')
print(len(pwd))
print(pwd)

40
7c4a8d09ca3762af61e59520943dc26494f8941b


### 其他摘要算法

In [8]:
import hashlib

a = "a test string".encode('utf-8')

res = hashlib.md5(a).hexdigest()
print('md5:{}, len= {}'.format(res, len(res)))
res = hashlib.sha1(a).hexdigest()
print('sha1:{}, len= {}'.format(res, len(res)))
res = hashlib.sha224(a).hexdigest()
print('sha224:{}, len= {}'.format(res, len(res)))
res = hashlib.sha256(a).hexdigest()
print('sha256:{}, len= {}'.format(res, len(res)))
res = hashlib.sha384(a).hexdigest()
print('sha384:{}, len= {}'.format(res, len(res)))
res = hashlib.sha512(a).hexdigest()
print('sha512:{}, len= {}'.format(res, len(res)))

md5:b1a4cf30d3f4095f0a7d2a6676bcae77, len= 32
sha1:2da75da5c85478df42df0f917700241ed282f599, len= 40
sha224:fb29c0daf4a995d2caad4cc5150a6977f871ac34c1c9c97d740d90d0, len= 56
sha256:b830543dc5d1466110538736d35c37cc61d32076a69de65c42913dfbb1961f46, len= 64
sha384:1b7c9d326961a90d6fe5e168b7339852a6238af953e5cad5501d140167bdfe8ebd82d7df5f7915ebad3e2f89d977357f, len= 96
sha512:fd308aadbf52384412c4ba3e2dfe3551e0faa2e7455898dae04fda4f238569e3889c56cbd4d120cf69f81a5f06456f327c19100eaed2e590888342f1ce3e0261, len= 128


## 加密与解密

In [24]:
import json
from Crypto.Cipher import AES

#秘钥,此处需要将字符串转为字节
key = b'0e9ec82fc54142de919f557a28bf637a'

#加密内容需要长达16位字符，所以进行空格拼接
def pad(text):
  while len(text) % 16 != 0:
    text += b' '
  return text

#加密秘钥需要长达16位字符，所以进行空格拼接
def pad_key(key):
  while len(key) % 16 != 0:
    key += b' '
  return key

#进行加密算法，模式ECB模式，把叠加完16位的秘钥传进来
aes = AES.new(pad_key(key), AES.MODE_ECB)
aa = {"action":"0","address":"","area":"","city":""}
#加密内容,此处需要将字符串转为字节
text=b'thisisjiamineirong'
# text = str(json.dumps(aa))
#进行内容拼接16位字符后传入加密类中，结果为字节类型
encrypted_text = aes.encrypt(pad(text))

print(encrypted_text)

#此处是为了验证是否能将字节转为字符串后，进行解密成功
#实际上a 就是 encrypted_text ，也就是加密后的内容
a = b'\x91\xbc\x7f*\xae\x02\xf8\x0eB\xc8\x98\x03z\xbb5@<(\xa1\x90\xad\x14veyn\x18\x18\xb5\xf7,\x9b'

#用aes对象进行解密，将字节类型转为str类型，错误编码忽略不计
de = str(aes.decrypt(a),encoding='utf-8',errors="ignore")

#获取str从0开始到文本内容的字符串长度。
print(de[:len(text)])


b'\x91\xbc\x7f*\xae\x02\xf8\x0eB\xc8\x98\x03z\xbb5@<(\xa1\x90\xad\x14veyn\x18\x18\xb5\xf7,\x9b'
thisisjiamineirong


In [2]:
import hashlib
from Crypto.Cipher import AES
from binascii import b2a_hex, a2b_hex
from pprint import pprint
import json

class Crypto():
    def __init__(self, key):
        tmp = hashlib.sha1(key).digest()
        tmp = hashlib.sha1(tmp).digest()[:16]
        self.key = tmp
        self.mode = AES.MODE_ECB

    # 加密函数，如果text不是16的倍数【加密文本text必须为16的倍数！】，那就补足为16的倍数
    def encrypt(self, text):
        cryptor = AES.new(self.key, self.mode)
        # 这里密钥key 长度必须为16（AES-128）、24（AES-192）、或32（AES-256）Bytes 长度.目前AES-128足够用
        length = 16
        count = len(text)

        if count % length != 0:
            add = length - (count % length)
        else:
            add = 0
        text = text + (b'\0' * add)
        self.ciphertext = cryptor.encrypt(text)
        # 因为AES加密时候得到的字符串不一定是ascii字符集的，输出到终端或者保存时候可能存在问题
        # 所以这里统一把加密后的字符串转化为16进制字符串
        return b2a_hex(self.ciphertext)

    # 解密后，去掉补足的空格用strip() 去掉
    def decrypt(self, text):
        cryptor = AES.new(self.key, self.mode)
        plain_text = cryptor.decrypt(a2b_hex(text))
        return plain_text.rstrip(b'\0')

    def decrypt_str(self, text):
        data = self.decrypt(text)
        print(data)
        text = data.decode().strip()

#         text = data.decode().strip().strip('\x06').strip('\x0c')
        return text
    
data = 'xxxxxx'

cryptor = Crypto(b'xxxxa')
info = cryptor.decrypt_str(data)
# pprint(info)

# pprint(json.loads(info))